# Bộ phân loại hình ảnh đơn giản

Notebook này hướng dẫn bạn cách phân loại hình ảnh bằng cách sử dụng một mạng nơ-ron đã được huấn luyện trước.

**Những gì bạn sẽ học:**
- Cách tải và sử dụng một mô hình đã được huấn luyện trước
- Tiền xử lý hình ảnh
- Dự đoán trên hình ảnh
- Hiểu về điểm số độ tin cậy

**Trường hợp sử dụng:** Nhận diện các đối tượng trong hình ảnh (như "mèo", "chó", "xe hơi", v.v.)

---


## Bước 1: Nhập các thư viện cần thiết

Hãy nhập các công cụ mà chúng ta cần. Đừng lo lắng nếu bạn chưa hiểu hết tất cả những điều này!


In [ ]:
# Core libraries
import numpy as np
from PIL import Image
import requests
from io import BytesIO

# TensorFlow for deep learning
try:
    import tensorflow as tf
    from tensorflow.keras.applications import MobileNetV2
    from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
    print("✅ TensorFlow loaded successfully!")
    print(f"   Version: {tf.__version__}")
except ImportError:
    print("❌ Please install TensorFlow: pip install tensorflow")

## Bước 2: Tải mô hình đã được huấn luyện trước

Chúng ta sẽ sử dụng **MobileNetV2**, một mạng nơ-ron đã được huấn luyện trên hàng triệu hình ảnh.

Đây được gọi là **Học chuyển giao** - sử dụng một mô hình mà người khác đã huấn luyện!


In [ ]:
print("📦 Loading pre-trained MobileNetV2 model...")
print("   This may take a minute on first run (downloading weights)...")

# Load the model
# include_top=True means we use the classification layer
# weights='imagenet' means it was trained on ImageNet dataset
model = MobileNetV2(weights='imagenet', include_top=True)

print("✅ Model loaded!")
print(f"   The model can recognize 1000 different object categories")

## Bước 3: Các Hàm Hỗ Trợ

Hãy tạo các hàm để tải và chuẩn bị hình ảnh cho mô hình của chúng ta.


In [ ]:
def load_image_from_url(url):
    """
    Load an image from a URL.
    
    Args:
        url: Web address of the image
        
    Returns:
        PIL Image object
    """
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img


def prepare_image(img):
    """
    Prepare an image for the model.
    
    Steps:
    1. Resize to 224x224 (model's expected size)
    2. Convert to array
    3. Add batch dimension
    4. Preprocess for MobileNetV2
    
    Args:
        img: PIL Image
        
    Returns:
        Preprocessed image array
    """
    # Resize to 224x224 pixels
    img = img.resize((224, 224))
    
    # Convert to numpy array
    img_array = np.array(img)
    
    # Add batch dimension (model expects multiple images)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Preprocess for MobileNetV2
    img_array = preprocess_input(img_array)
    
    return img_array


def classify_image(img):
    """
    Classify an image and return top predictions.
    
    Args:
        img: PIL Image
        
    Returns:
        List of (class_name, confidence) tuples
    """
    # Prepare the image
    img_array = prepare_image(img)
    
    # Make prediction
    predictions = model.predict(img_array, verbose=0)
    
    # Decode predictions to human-readable labels
    # top=5 means we get the top 5 most likely classes
    decoded = decode_predictions(predictions, top=5)[0]
    
    # Convert to simpler format
    results = [(label, float(confidence)) for (_, label, confidence) in decoded]
    
    return results


print("✅ Helper functions ready!")

## Bước 4: Kiểm tra trên các hình ảnh mẫu

Hãy thử phân loại một số hình ảnh từ internet!


In [ ]:
# Sample images to classify
# These are from Unsplash (free stock photos)
test_images = [
    {
        "url": "https://images.unsplash.com/photo-1514888286974-6c03e2ca1dba?w=400",
        "description": "A cat"
    },
    {
        "url": "https://images.unsplash.com/photo-1552053831-71594a27632d?w=400",
        "description": "A dog"
    },
    {
        "url": "https://images.unsplash.com/photo-1511919884226-fd3cad34687c?w=400",
        "description": "A car"
    },
]

print(f"🧪 Testing on {len(test_images)} images...")
print("=" * 70)

### Phân loại Mỗi Hình ảnh


In [ ]:
for i, img_data in enumerate(test_images, 1):
    print(f"\n📸 Image {i}: {img_data['description']}")
    print("-" * 70)
    
    try:
        # Load image
        img = load_image_from_url(img_data['url'])
        
        # Display image
        display(img.resize((200, 200)))  # Show smaller version
        
        # Classify
        results = classify_image(img)
        
        # Show predictions
        print("\n🎯 Top 5 Predictions:")
        for rank, (label, confidence) in enumerate(results, 1):
            # Create a visual bar
            bar_length = int(confidence * 50)
            bar = "█" * bar_length
            
            print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
        
    except Exception as e:
        print(f"❌ Error: {e}")

print("\n" + "=" * 70)

## Bước 5: Thử với hình ảnh của bạn!

Thay thế URL bên dưới bằng bất kỳ URL hình ảnh nào bạn muốn phân loại.


In [ ]:
# Try your own image!
# Replace this URL with any image URL
custom_image_url = "https://images.unsplash.com/photo-1472491235688-bdc81a63246e?w=400"  # A flower

print("🖼️  Classifying your custom image...")
print("=" * 70)

try:
    # Load and show image
    img = load_image_from_url(custom_image_url)
    display(img.resize((300, 300)))
    
    # Classify
    results = classify_image(img)
    
    # Show results
    print("\n🎯 Top 5 Predictions:")
    print("-" * 70)
    for rank, (label, confidence) in enumerate(results, 1):
        bar_length = int(confidence * 50)
        bar = "█" * bar_length
        print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
    
    # Highlight top prediction
    top_label, top_confidence = results[0]
    print("\n" + "=" * 70)
    print(f"\n🏆 Best guess: {top_label} ({top_confidence*100:.2f}% confident)")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("   Make sure the URL points to a valid image!")

## 💡 Điều Gì Vừa Xảy Ra?

1. **Chúng ta đã tải một mô hình được huấn luyện trước** - MobileNetV2 đã được huấn luyện trên hàng triệu hình ảnh
2. **Chúng ta đã tiền xử lý hình ảnh** - Thay đổi kích thước và định dạng chúng phù hợp với mô hình
3. **Mô hình đã đưa ra dự đoán** - Nó xuất ra xác suất cho 1000 lớp đối tượng
4. **Chúng ta đã giải mã kết quả** - Chuyển đổi các con số thành nhãn dễ hiểu

### Hiểu Về Điểm Tự Tin

- **90-100%**: Rất tự tin (gần như chắc chắn đúng)
- **70-90%**: Tự tin (có thể đúng)
- **50-70%**: Tương đối tự tin (có thể đúng)
- **Dưới 50%**: Không tự tin lắm (không chắc chắn)

### Tại sao dự đoán có thể sai?

- **Góc hoặc ánh sáng bất thường** - Mô hình được huấn luyện trên các bức ảnh thông thường
- **Nhiều đối tượng** - Mô hình mong đợi một đối tượng chính
- **Đối tượng hiếm** - Mô hình chỉ biết 1000 danh mục
- **Hình ảnh chất lượng thấp** - Hình ảnh mờ hoặc bị vỡ pixel sẽ khó hơn


## 🚀 Các bước tiếp theo

1. **Thử các hình ảnh khác:**
   - Tìm hình ảnh trên [Unsplash](https://unsplash.com)
   - Nhấp chuột phải → "Sao chép địa chỉ hình ảnh" để lấy URL

2. **Thử nghiệm:**
   - Điều gì xảy ra với nghệ thuật trừu tượng?
   - Nó có thể nhận diện các đối tượng từ các góc độ khác nhau không?
   - Nó xử lý nhiều đối tượng như thế nào?

3. **Tìm hiểu thêm:**
   - Khám phá [bài học về Thị giác Máy tính](../lessons/4-ComputerVision/README.md)
   - Học cách huấn luyện bộ phân loại hình ảnh của riêng bạn
   - Hiểu cách hoạt động của CNNs (Mạng Nơ-ron Tích chập)

---

## 🎉 Chúc mừng!

Bạn vừa xây dựng một bộ phân loại hình ảnh sử dụng mạng nơ-ron tiên tiến!

Kỹ thuật này đang được ứng dụng trong:
- Google Photos (sắp xếp ảnh của bạn)
- Xe tự lái (nhận diện đối tượng)
- Chẩn đoán y tế (phân tích hình ảnh X-quang)
- Kiểm soát chất lượng (phát hiện lỗi)

Tiếp tục khám phá và học hỏi nhé! 🚀



---

**Tuyên bố miễn trừ trách nhiệm**:  
Tài liệu này đã được dịch bằng dịch vụ dịch thuật AI [Co-op Translator](https://github.com/Azure/co-op-translator). Mặc dù chúng tôi cố gắng đảm bảo độ chính xác, xin lưu ý rằng các bản dịch tự động có thể chứa lỗi hoặc không chính xác. Tài liệu gốc bằng ngôn ngữ bản địa nên được coi là nguồn thông tin chính thức. Đối với các thông tin quan trọng, khuyến nghị sử dụng dịch vụ dịch thuật chuyên nghiệp bởi con người. Chúng tôi không chịu trách nhiệm cho bất kỳ sự hiểu lầm hoặc diễn giải sai nào phát sinh từ việc sử dụng bản dịch này.
